In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import os
import glob as gb
import cv2
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from torchsummary import summary



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Hyperparameters
in_channels = 3
num_classes = 4
learning_rate = 0.001
batch_size = 64
# Load Data
trainpath ='/content/drive/MyDrive/covid19/pytorch2/train/'
testpath='/content/drive/MyDrive/covid19/pytorch2/test/'
valpath='/content/drive/MyDrive/covid19/pytorch2/val/'

img_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize(size=(224,224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]), 
        ]),

    'test':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]), 
        ]),
        'val':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]), 
        ]),
     }

train_dataset=datasets.ImageFolder(trainpath,transform=img_transforms['train'])
test_dataset=datasets.ImageFolder(testpath,transform=img_transforms['test'])
val_dataset=datasets.ImageFolder(testpath,transform=img_transforms['val'])
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)


it=iter(train_loader)
images,label=it.next()
print(images.shape)
print(label.shape)



torch.Size([64, 3, 224, 224])
torch.Size([64])


In [4]:
# Download and load the pretrained ResNet-50.
from torchvision.models import resnet50, ResNet50_Weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights = weights)
# finetune only the top layer of the model.
for param in model.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
model.fc = nn.Linear(model.fc.in_features, 4)  
# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = model(images)
print (outputs.size())     

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

torch.Size([64, 4])


In [5]:
#due to session crach i didnt use validation accuracy in train step
def train(num_epochs,model,learning_rate,train_loader):

# Loss and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  trainingloss = []
  validationloss = []
# Train the model
  total_step = len(train_loader)
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
              
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    #model.eval()  
    #with torch.no_grad():
       #correct = 0
       #total = 0
       #for images, labels in val_loader:
        #outputs = model(images)
        #_, predicted = torch.max(outputs.data, 1)
        #total += labels.size(0)
        #correct += (predicted == labels).sum().item()
    #acc=(100 * correct / total)    
          # printing training/validation statistics 
    #print('Epoch: {}  \tValidation acc: {:.6f}'.format(epoch,acc))

In [ ]:
num_epochs=4
learning_rate=0.01
train(num_epochs,model,learning_rate,train_loader)

In [ ]:
# Test the model
def test(model,test_loader): 

  model.eval()  
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

  print('Test Accuracy of the model is: {} %'.format(100 * correct / total))


In [ ]:
test(model,test_loader)

Test Accuracy of the model is: 85.21739130434783 %


In [ ]:
#efficient net
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights)
for param in model.features.parameters():
    param.requires_grad = False
# Replace the top layer for finetuning.
model.fc = nn.Linear(512, 4)
# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = model(images)
print (outputs.size()) 

torch.Size([64, 1000])


In [ ]:
num_epochs=4
learning_rate=0.01
train(num_epochs,model,learning_rate,train_loader)

In [ ]:
test(model,test_loader)

Test Accuracy of the model is: 86.08695652173913 %


In [ ]:
#efficient net B1
weights = torchvision.models.EfficientNet_B1_Weights.DEFAULT
model = torchvision.models.efficientnet_b1(weights=weights)
for param in model.features.parameters():
    param.requires_grad = False
# Replace the top layer for finetuning.
model.fc = nn.Linear(512, 4)
# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = model(images)
print (outputs.size()) 

Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth


  0%|          | 0.00/30.1M [00:00<?, ?B/s]

torch.Size([64, 1000])


In [ ]:
num_epochs=4
learning_rate=0.01
train(num_epochs,model,learning_rate,train_loader)

In [ ]:
test(model,test_loader)

Test Accuracy of the model is: 82.82608695652173 %
